In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 18.8 MB/s 
     |████████████████████████████████| 7.6 MB 69.7 MB/s 
     |████████████████████████████████| 163 kB 75.3 MB/s 


In [2]:
import torch
from transformers import Trainer, TrainingArguments, DefaultFlowCallback, AdamW, get_constant_schedule
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
import json
def read_json(input_file):
    """Reads a json list file."""
    with open(input_file, "r") as f:
        reader = f.readlines()
    return [json.loads(line.strip()) for line in reader]

In [3]:
from transformers import BertTokenizerFast, BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('Langboat/mengzi-bert-base', num_labels=36)
tokenizer = BertTokenizerFast.from_pretrained('Langboat/mengzi-bert-base')

Downloading:   0%|          | 0.00/849 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/206M [00:00<?, ?B/s]

Some weights of the model checkpoint at Langboat/mengzi-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'sop.cls.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'sop.cls.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df = pd.DataFrame.from_records(read_json('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/data/train.json'))
# df1 = df
# df = pd.concat([df,df1])
# df = df.sample(frac=1,random_state=42)
# df.shape[0]

In [6]:
df.head()

,id,title,assignee,abstract,label_id
0,538f267d2e6fba48b1286fb7f1499fe7,一种信号的发送方法及基站、用户设备,华为技术有限公司,一种信号的发送方法及基站、用户设备。在一个子帧中为多个用户设备配置的参考信号的符号和数据的符...,0
1,635a7d4b6358b6ff24a324bb871505db,一种5G通讯电缆故障监控系统,中铁二十二局集团电气化工程有限公司,本发明公开了一种5G通讯电缆故障监控系统，包括信号采样模块、补偿反馈模块，所述信号采样模块对...,0
2,aaf98d6bfe1932cf1a262812ca59d1ba,一种测试方法及电子设备,腾讯科技(北京)有限公司,本发明提供了一种测试方法及电子设备，该方法包括：基于选取的测试任务确定目标测试用例，根据所述...,0
3,ad49c9ba6412452d9b25071af702f4fb,天线方位角调节装置,武汉虹信通信技术有限责任公司,一种天线方位角调节装置，包括对向的两个8字形支架(101)、动力输入电机(102)、主动齿轮...,0
4,ffa2d7617b3eac3a1d7df01e5bb515a2,光纤老化预测方法及装置,新华三大数据技术有限公司,本申请提供一种光纤老化预测方法及装置，所述方法包括：获取待测光纤模块可接收的光信号的告警阈值...,0


In [7]:
df['input_string'] = df.apply(lambda x: f"这份专利的标题为：《{x.title}》，由“{x.assignee}”公司申请，详细说明如下：{x.abstract}",axis=1)
df = df[['input_string','label_id']]

In [8]:
df.head()

,input_string,label_id
0,这份专利的标题为：《一种信号的发送方法及基站、用户设备》，由“华为技术有限公司”公司申请，详...,0
1,这份专利的标题为：《一种5G通讯电缆故障监控系统》，由“中铁二十二局集团电气化工程有限公司”...,0
2,这份专利的标题为：《一种测试方法及电子设备》，由“腾讯科技(北京)有限公司”公司申请，详细说...,0
3,这份专利的标题为：《天线方位角调节装置》，由“武汉虹信通信技术有限责任公司”公司申请，详细说...,0
4,这份专利的标题为：《光纤老化预测方法及装置》，由“新华三大数据技术有限公司”公司申请，详细说...,0


In [9]:
df_train, df_dev = train_test_split(df, test_size=0.2, random_state=42)
df_train = df_train.reset_index()
df_dev = df_dev.reset_index()

In [10]:
df_train.head()

,index,input_string,label_id
0,302,这份专利的标题为：《基于电量变化的共享单车计费方法和系统、以及用车方法》，由“南京昱立信息科...,4
1,467,这份专利的标题为：《双光子荧光材料苯并噁唑基吡啶盐及其制备方法和应用》，由“安徽大学”公司申...,8
2,294,这份专利的标题为：《一种基于区块链技术的商品交易信息共享方法》，由“河钢国际科技(北京)有限...,4
3,548,这份专利的标题为：《聚光模组、光源组件及液相检测器》，由“山东悟空仪器有限公司”公司申请，详...,10
4,465,这份专利的标题为：《一种套管式三腔二囊管牵引装置》，由“中国人民解放军第四军医大学”公司申请...,8


In [ ]:
from torch.utils.data import Dataset
class BertDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.input_encodings = tokenizer([data.loc[i,'input_string'] for i in range(len(data))], 
                                         max_length=512, 
                                         truncation=True, 
                                         padding=True)
        self.targets = data['label_id'].astype(int)
    def __len__(self):
        return len(self.targets)
    def __getitem__(self, index):
        return {
            'input_ids': torch.tensor(self.input_encodings['input_ids'][index]),
            'token_type_ids': torch.tensor(self.input_encodings['token_type_ids'][index]),
            'attention_mask': torch.tensor(self.input_encodings['attention_mask'][index]),
            'labels': torch.tensor(self.targets[index])
        }
train_dataset = BertDataset(df_train, tokenizer)
dev_dataset = BertDataset(df_dev, tokenizer)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(preds, labels, average='macro')
    print('f1: ', f1)
    return {'f1': f1}
training_args = TrainingArguments(
    output_dir=f'./ckpts',
    num_train_epochs=40,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=32,   # 2*per_device_train_batch_size
    dataloader_num_workers=8,
    logging_dir=f'./logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    fp16=False,
    save_total_limit=10,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to=[],
)
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=True)
trainer = Trainer(
    model=model, 
    args=training_args,     
    train_dataset=train_dataset,
    eval_dataset=dev_dataset, 
    optimizers= (optimizer, get_constant_schedule(optimizer, last_epoch=-1)),
    compute_metrics=compute_metrics
)
trainer.train()
trainer.save_model(f'./best_ckpt/')

In [ ]:
model = BertForSequenceClassification.from_pretrained('./best_ckpt/', num_labels=36)
model = model.to('cuda')
tokenizer = BertTokenizerFast.from_pretrained('Langboat/mengzi-bert-base')
df_test = pd.DataFrame.from_records(read_json('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/data/testA.json'))
df_test['input_string'] = df_test.apply(lambda x: f"这份专利的标题为：《{x.title}》，由“{x.assignee}”公司申请，详细说明如下：{x.abstract}",axis=1)
BATCH_SIZE = 6
total_batch_num = int(len(df_test)/BATCH_SIZE) + 1
outputs_list = []
for batch_num in tqdm(range(total_batch_num)):
    batch_input = df_test[batch_num*BATCH_SIZE:(batch_num+1)*BATCH_SIZE].reset_index(drop=True)
    if len(batch_input)==0:
        break
    tokens = tokenizer([batch_input.loc[i,'input_string'] for i in range(len(batch_input))], 
                         return_tensors="pt",
                         max_length=512, 
                         truncation=True, 
                         padding=True).to('cuda')
    outputs = model(input_ids=tokens.input_ids,
                    attention_mask=tokens.attention_mask,
                    token_type_ids=tokens.token_type_ids)
    res_tmp = outputs['logits'].argmax(1).tolist()
    outputs_list.extend(res_tmp)
    del tokens, res_tmp
assert len(df_test) == len(outputs_list)
df_test['label'] = outputs_list
df_test['label'] = df_test['label'].apply(int)
df_test[['id','label']].to_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/submission.csv',index=False)